#**LOGISTIC REGRESSION**

#**I. Import Libraries**

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np

In [ ]:
from sklearn.model_selection import train_test_split
from skimage.feature import hog

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#**II. Load Folio Data**

In [ ]:
!git clone https://github.com/erwin24092002/Datasets

fatal: destination path 'Datasets' already exists and is not an empty directory.


In [ ]:
annot = os.listdir('Datasets/folio_leaf')
annot_id = list(range(len(annot)))
print(annot)
print(annot_id)

['caricature plant', 'barbados cherry', 'jackfruit', 'croton', 'betel', 'bitter orange', 'ashanti blood', 'coffee', 'coeur demoiselle', 'mulberry leaf']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [ ]:
X_train = []
X_test = []
y_train = []
y_test = []

In [ ]:
path = 'Datasets/folio_leaf'
for x in annot_id:
  x_path = os.path.join(path, annot[x])
  img_name = os.listdir(x_path)
  X = []
  y = []

  for img in img_name:
    image = cv2.imread(os.path.join(x_path, img), 0)
    X.append(cv2.resize(image, (int(image.shape[1]*0.1), int(image.shape[0]*0.1))))
    y.append(x)

  X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(X, y, test_size=0.3)
  X_train.extend(X_train_t)
  X_test.extend(X_test_t)
  y_train.extend(y_train_t)
  y_test.extend(y_test_t)

#**III. Extracting Feature**

##**1. Convert_to_1D**

In [ ]:
def convert_to_1D(image):
  return np.reshape(image, image.shape[0]*image.shape[1])

In [ ]:
X_train_1D = []
for x in X_train:
  X_train_1D.append(convert_to_1D(x))
  
X_test_1D = []
for x in X_test:
  X_test_1D.append(convert_to_1D(x))

##**2. Histogram of Oriented Gradients**

In [ ]:
def calculate_hog(image):
  return hog(image, orientations=9, pixels_per_cell=(4,4), 
             cells_per_block=(2,2), block_norm='L2')

In [ ]:
X_train_hog = []
for x in X_train:
  X_train_hog.append(calculate_hog(x))

X_test_hog = []
for x in X_test:
  X_test_hog.append(calculate_hog(x))

#**IV. Training Model**

1D_Model

In [ ]:
t1D_model = LogisticRegression(max_iter=10000, random_state=1, n_jobs=2)
t1D_model.fit(X_train_1D, y_train) 

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=2, penalty='l2', random_state=1,
                   solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

HOG_Model

In [ ]:
HOG_model = LogisticRegression(max_iter=10000, random_state=1, n_jobs=2)
HOG_model.fit(X_train_hog, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=2, penalty='l2', random_state=1,
                   solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

#**V. Evaluating**

Result

In [ ]:
y_pred_1D = t1D_model.predict(X_test_1D)
y_pred_hog = HOG_model.predict(X_test_hog)

Evaluating

In [ ]:
arr1 = np.array([[accuracy_score(y_test, y_pred_1D), 
                precision_score(y_test, y_pred_1D, average='micro'), 
                recall_score(y_test, y_pred_1D, average='micro'), 
                f1_score(y_test, y_pred_1D, average='micro')]],
               dtype='float32')
arr2 = np.array([[accuracy_score(y_test, y_pred_hog), 
                 precision_score(y_test, y_pred_hog, average='micro'), 
                 recall_score(y_test, y_pred_hog, average='micro'), 
                 f1_score(y_test, y_pred_hog, average='micro')]],
               dtype='float32')
arr = np.concatenate((arr1, arr2), axis=0)

arr_df = pd.DataFrame(data=arr, index=['1D MODEL', 'HOG MODEL'], 
                      columns=['ACCURACY', 'PRECISION', 'RECALL', 'F1'])
pd.options.display.float_format = '{:.3f}'.format
arr_df


,ACCURACY,PRECISION,RECALL,F1
1D MODEL,0.900,0.900,0.900,0.900
HOG MODEL,0.867,0.867,0.867,0.867
